In [2]:
import requests

In [3]:
URL = "https://thor-galabau.com/"

In [4]:
# request html content
html = requests.get(URL).content

In [65]:
# read the text of the whole page
from bs4 import BeautifulSoup
soup = BeautifulSoup(html)
body_raw = soup.get_text()

In [66]:
# format the text
import re
body_raw = re.compile(r'\n').sub(' ', body_raw)
body_raw = re.compile(r'\t').sub(' ', body_raw)
body_text = body_raw.strip()

In [67]:
body_text

'THOR Garten- & Landschaftsbau • Experten für Ihre Gartengestaltung.                                                                   Leistungen Über uns Inspirationen Karriere Kontakt    Menü  Leistungen Über uns Inspirationen Karriere Kontakt                                                           THOR Garten- & Landschaftsbau     Ihr Gartenbau-Partner im Rhein-Main-Gebiet.     Der grüne Hammer für Ihren Garten.                 Was bieten wir Ihnen im Garten- und Landschaftsbau?     Als Experte für Garten- und Landschaftsbau planen, gestalten und setzen wir Ihre Gartenträume in die Wirklichkeit um. Wir sind für Sie der ideale Partner in Frankfurt und dem kompletten Rhein-Main-Gebiet voller Erfahrung und Fachwissen rund um das Thema Gartengestaltung und Gartenbau.Wir kennen die besonderen Eigenschaften und Bedürfnisse der verschiedenen Blumen, Gräser, Stauden, Sträucher und Bäume, die wir gemeinsam für Ihren Garten auswählen und fachgerecht am richtigen Standort verpflanzen.Als reg

In [68]:
import os
os.chdir('/Users/siegstedt/projects/kreativkopf')

In [69]:
from src.prepare import generate_keywords_from_text
number_of_keywords = 5
keyword_tuples = generate_keywords_from_text(body_text, number_of_keywords)
keyword_tuples

[('garten', 0.20772288200382844),
 ('gartenbau', 0.201765862877595),
 ('gartengestaltung', 0.1409032267985325),
 ('leistungen', 0.1409032267985325),
 ('kontakt', 0.1402207255035673)]

In [14]:
keyword_list = [i for i,j in keyword_tuples]
keyword_list

['garten', 'gartenbau', 'gartengestaltung', 'leistungen', 'kontakt']

In [15]:
# explicit check for locations in text
for i in ['Frankfurt', 'Mainhausen']:
    if i in body_text:
        print('Found location:',i)

Found location: Frankfurt
Found location: Mainhausen


In [5]:
# retrieve childpages from homepage
from scrapy import Selector

html = requests.get(URL).content
sel = Selector(text=html)

In [6]:
# select all links no the first page
links = sel.xpath('//a/@href').extract()

In [7]:
# deduplicate
links = sorted(list(set(links)))
links

['https://thor-galabau.com',
 'https://thor-galabau.com/datenschutz/',
 'https://thor-galabau.com/impressum/',
 'https://thor-galabau.com/inspirationen/',
 'https://thor-galabau.com/karriere/',
 'https://thor-galabau.com/karriere/#jobs',
 'https://thor-galabau.com/kontakt/',
 'https://thor-galabau.com/leistungen/',
 'https://thor-galabau.com/leistungen/#gartenarbeiten',
 'https://thor-galabau.com/leistungen/#gartenbau',
 'https://thor-galabau.com/leistungen/#gartengestaltung',
 'https://thor-galabau.com/leistungen/#gartenpflege',
 'https://thor-galabau.com/leistungen/#pflasterarbeiten',
 'https://thor-galabau.com/leistungen/#tiefbau',
 'https://thor-galabau.com/ueber-uns/',
 'https://www.facebook.com/THOR.GaLaBau',
 'https://www.instagram.com/thor.galabau/',
 'https://www.linkedin.com/company/thor-galabau',
 'mailto:info@thor-galabau.com',
 'tel:+4961828249250']

In [19]:
# clear noise
links = [
    i for 
    i in links 
    if 
        (not '/#' in i) # clear all page references
        or (i[:4] == 'http') # clear all non-http links
        or (not i == URL) # clear references to home page
]

In [20]:
links

['https://thor-galabau.com',
 'https://thor-galabau.com/datenschutz/',
 'https://thor-galabau.com/impressum/',
 'https://thor-galabau.com/inspirationen/',
 'https://thor-galabau.com/karriere/',
 'https://thor-galabau.com/kontakt/',
 'https://thor-galabau.com/leistungen/',
 'https://thor-galabau.com/ueber-uns/',
 'https://www.facebook.com/THOR.GaLaBau',
 'https://www.instagram.com/thor.galabau/',
 'https://www.linkedin.com/company/thor-galabau']

In [22]:
# extract social media links
facebook_links = [i for i in links if 'facebook' in i]
instagram_links = [i for i in links if 'instagram' in i]
linkedin_links = [i for i in links if 'linkedin' in i]

In [23]:
# drop social media links
childpages = [i for i in links if i not in socialmedia_links]

In [77]:
include_socialmedia = True

if include_socialmedia:
    outdict = [
        {"childpages": childpages},
        {"facebook_links": facebook_links},
        {"instagram_links": instagram_links},
        {"linkedin_links": linkedin_links},
    ]
else:
    outdict = [
        {"childpages": childpages},    
    ]
outdict

[{'childpages': ['https://thor-galabau.com',
   'https://thor-galabau.com/datenschutz/',
   'https://thor-galabau.com/impressum/',
   'https://thor-galabau.com/inspirationen/',
   'https://thor-galabau.com/karriere/',
   'https://thor-galabau.com/kontakt/',
   'https://thor-galabau.com/leistungen/',
   'https://thor-galabau.com/ueber-uns/']},
 {'facebook_links': ['https://www.facebook.com/THOR.GaLaBau']},
 {'instagram_links': ['https://www.instagram.com/thor.galabau/']},
 {'linkedin_links': ['https://www.linkedin.com/company/thor-galabau']}]